In [18]:
from andi_datasets.models_phenom import models_phenom
from andi_datasets.utils_videos import transform_to_video
from andi_datasets.utils_trajectories import motion_blur
from helpersMSD import *
from helpersGeneration import *
from helpersPlot import *
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# number of time steps per trajectory (frames)
T = 200
# number of trajectories
N = 1
# Length of box (pixels), set to 0 to make them start at 0,0
L = 0 
# diffusion coefficient (pixels^2 / frame)
D = 0.1

In [6]:
trajs, labels = models_phenom().single_state(N, 
                                L = L,
                                T = T,
                                Ds = [D, 0.0], # Mean and variance
                                alphas = 1)
print(trajs.shape,labels.shape)
# Need to reshape generated trajectories because they are in format (T,N,dim), but we want them in (N,T,dim)
trajs = trajs.transpose(1,0,2)
labels = labels.transpose(1,0,2)
print(trajs.shape,labels.shape)
# Labels are now in format (N,T,3): for each particle T times the tuple (alpha, D, state)
print(f"Particle 0 [alpha, Gen_D, state]: {labels[0,0,:]}")


(200, 1, 2) (200, 1, 3)
(1, 200, 2) (1, 200, 3)
Particle 0 [alpha, Gen_D, state]: [1.  0.1 2. ]


In [ ]:
nposframe = 10


%load_ext autoreload
%reload_ext autoreload
%autoreload 2
videos = trajectories_to_video(trajs,nposframe,image_props={"upsampling_factor":10,
                                            #"background_intensity": [0,0],
                                            "particle_intensity": [500,20],
                                            #"resolution": 200e-9
                                              })
print(np.max(videos))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
748.5070576393385


In [ ]:
print(videos.shape)
play_video(videos[0,:,:,:])


INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


(1, 20, 32, 32)
vmin: 62.97557623402027 vmax: 748.5070576393385 mean: 103.38


INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


vmin: 63.10955103806469 vmax: 427.0343150895168 mean: 100.97


In [ ]:
oversamp_factor = 10
output_length = T//oversamp_factor
exposure_time = 0.2

MB = motion_blur(output_length = output_length, oversamp_factor = oversamp_factor, exposure_time = exposure_time)

trajsTest, labelsTest = models_phenom().single_state(N, 
                                L = L,
                                T = T,
                                Ds = [D, 0.0], # Mean and variance
                                alphas = 1)
print(trajsTest.shape,labelsTest.shape)
# Need to reshape generated trajectories because they are in format (T,N,dim), but we want them in (N,T,dim)
trajsT = trajsTest.transpose(1,0,2)
labelsT = labelsTest.transpose(1,0,2)
print(trajsT.shape,labelsT.shape)
# Labels are now in format (N,T,3): for each particle T times the tuple (alpha, D, state)
print(f"Particle 0 [alpha, Gen_D, state]: {labelsT[0,0,:]}")

video = transform_to_video(
    trajsTest,
    motion_blur_generator=MB,
    optics_props={ "output_region": [-16, -16, 16, 16]
                  },
    background_props = {
        "background_mean": 50,  # Mean background intensity
        "background_std": 0,  # Standard deviation of background intensity within a video
    }

)
print(video.shape)

videos2= trajectories_to_video(trajsT,nposframe,image_props={"upsampling_factor":5,
                                            "background_intensity": [200,20],
                                            "particle_intensity": [500,20]
                                              })
print(videos2.shape)

(200, 1, 2) (200, 1, 3)
(1, 200, 2) (1, 200, 3)
Particle 0 [alpha, Gen_D, state]: [1.  0.1 2. ]
(20, 32, 32, 1)
(1, 20, 32, 32)


In [35]:
play_video(video)
play_video(videos2[0,:,:,:])

# Slight difference between the andi-dataset videos and our version.
# The andi version has a much higher background value, we can put our background higher to compensate for this

INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


vmin: 3610.0 vmax: 5418.0 mean: 3870.39


INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


vmin: 120.05667801934669 vmax: 498.83468932074175 mean: 200.62


In [ ]:
#%timeit video = transform_to_video(trajsTest,motion_blur_generator=MB,optics_props={ "output_region": [-16, -16, 16, 16]},background_props = {"background_mean": 50,  "background_std": 0, })
trajsTime, _ = models_phenom().single_state(1000, 
                                L = L,
                                T = T,
                                Ds = [D, 0.0], # Mean and variance
                                alphas = 1)
trajsTime = trajsTime.transpose(1,0,2)


%timeit videos = trajectories_to_video(trajs,nposframe,{"upsampling_factor":5,"poisson_noise":-1 })
%timeit videos = trajectories_to_video(trajs,nposframe,{"upsampling_factor":5,"poisson_noise":1  })

#%timeit videos2 = trajectories_to_video2(trajs,nposframe,{"upsampling_factor":5, })

54.6 ms ± 760 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
52.8 ms ± 995 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
videosCentered= trajectories_to_video(trajsT,nposframe,
                                              #center=True,
                                              image_props={"upsampling_factor":5,
                                            "background_intensity": [200,20],
                                            "particle_intensity": [500,20],
                                            "poisson_noise":-1,
                                              })

videosUncentered= trajectories_to_video(trajsT,nposframe,
                                        image_props={"upsampling_factor":5,
                                            "background_intensity": [200,20],
                                            "particle_intensity": [500,20],
                                            "poisson_noise":100,
                                              })

print(videos.shape)
play_video(videosCentered[0,:,:,:])
play_video(videosUncentered[0,:,:,:])

INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


(1, 20, 32, 32)
vmin: 126.32622684408459 vmax: 928.8171253709731 mean: 203.39


INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


vmin: 115.28 vmax: 987.76 mean: 203.51
